<a href="https://colab.research.google.com/github/zwcrowley/module_21_deep_learning_challenge/blob/main/deep_learning_charity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 4: Predicting Congressional Bill Passage**

# Machine Learning Optimization and Model Output

## Team 7


## Import dependencies and read in data:

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#  Import and read the cleaned house data from AWS S3:
# house_df = pd.read_csv("https://project-4-team7.s3.ca-central-1.amazonaws.com/cleaned_house.csv")
# house_df.head()

In [50]:
# Import from google drive folder:
# Mount google drive to get data:
from google.colab import drive
drive.mount('/content/gdrive')
house_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/Resources/house_cleaned.csv"
senate_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/Resources/senate_cleaned.csv"

# Read in house and senatedata using pandas:
house_df = pd.read_csv(house_filepath)
senate_df = pd.read_csv(senate_filepath)
# Glimpse house data:
house_df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,Agriculture,Appropriations,Armed Services,...,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VI,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,113,200,200,0,0,46,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,113,179,179,0,0,42,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,113,0,0,0,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,113,0,0,0,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,113,200,197,3,0,46,10,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Building two ml models, one with House of Reps data and one with Senate Data

# House Model:

## Preprocessing

In [7]:
# Check for NAs, duplicates and get the shape of the data:
print(f'The shape of the house_df data is: {house_df.shape}')
print(f'The number of NAs in the house_df data: {house_df.isnull().sum()}')
print(f'The duplicate rows of NAs in the house_df data: {house_df.duplicated().sum()}')
# There are 41 columns, 40683 rows, and no NAs or duplicates in the house_df dataset.
# target is bill_passed

The shape of the application data is: (40683, 41)
The number of NAs in the application data: Bill Type                              0
Congress                               0
Number of Cosponsors                   0
Cosponsor Dems                         0
Cosponsor Reps                         0
Cosponsor Ind                          0
Cosponsor States                       0
Committees                            49
Latest Action                         30
Subject                              729
Sponsor Title                         16
Sponsor Party                         16
Sponsor State                         16
Month Introduced                       0
Agriculture                            0
Appropriations                         0
Armed Services                         0
Budget                                 0
Education and the Workforce            0
Energy and Commerce                    0
Ethics                                 0
Financial Services                     0
Forei

In [9]:
# Numeric variable stats
house_df.describe()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,Agriculture,Appropriations,Armed Services,...,Transportation and Infrastructure,Veterans' Affairs,Ways and Means,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
count,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,...,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000,40683.000000
mean,115.362805,15.790330,9.883711,5.906398,0.000172,7.205049,5.376472,0.042843,0.016837,0.053438,...,0.077452,0.057985,0.192242,0.009857,0.001204,0.001204,0.001204,0.001204,0.028980,0.115847
std,1.470997,34.220849,25.162760,17.237882,0.013116,9.655811,3.294271,0.202507,0.128664,0.224907,...,0.267311,0.233718,0.394067,0.098792,0.034684,0.034684,0.034684,0.034684,0.167753,0.320045
min,113.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,116.000000,4.000000,1.000000,1.000000,0.000000,3.000000,5.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,117.000000,15.000000,7.000000,4.000000,0.000000,10.000000,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,118.000000,386.000000,238.000000,244.000000,1.000000,58.000000,12.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
# Get the column names:
list(house_df.columns)


['Bill Type',
 'Congress',
 'Number of Cosponsors',
 'Cosponsor Dems',
 'Cosponsor Reps',
 'Cosponsor Ind',
 'Cosponsor States',
 'Committees',
 'Latest Action',
 'Subject',
 'Sponsor Title',
 'Sponsor Party',
 'Sponsor State',
 'Month Introduced',
 'Agriculture',
 'Appropriations',
 'Armed Services',
 'Budget',
 'Education and the Workforce',
 'Energy and Commerce',
 'Ethics',
 'Financial Services',
 'Foreign Affairs',
 'Homeland Security',
 'House Administration',
 'Judiciary',
 'Natural Resources',
 'Oversight and Accountability',
 'Rules',
 'Science, Space, and Technology',
 'Small Business',
 'Transportation and Infrastructure',
 "Veterans' Affairs",
 'Ways and Means',
 'Intelligence',
 'Printing',
 'Taxation',
 'Library',
 'Economic',
 'bill_passed',
 'bill_referred_committee']

### Model Target:


*   "Latest Action" == "Became Public Law" is the target for the models



In [13]:
# Model Target:
house_df["bill_passed"].value_counts()
# 1179 bills  in the dataset which originated from the House became law in the 113th-118th Congresses

0    39504
1     1179
Name: bill_passed, dtype: int64

In [14]:
# Drop the non-beneficial columns: 'Unnamed' column.
house_df = house_df.drop(["Bill Type", "Committees", "Latest Action" ], axis='columns')
house_df.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Subject,Sponsor Title,Sponsor Party,Sponsor State,...,Transportation and Infrastructure,Veterans' Affairs,Ways and Means,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,113,200,200,0,0,46,Accounting and auditing,Rep.,D,WI,...,0,0,0,0,0,0,0,0,0,0
1,113,179,179,0,0,42,Administrative law and regulatory procedures,Rep.,D,GA,...,0,1,0,0,0,0,0,0,0,0
2,113,0,0,0,0,0,NaN,Rep.,D,CA,...,1,1,1,1,1,1,1,1,1,1
3,113,0,0,0,0,0,NaN,Rep.,D,CA,...,1,1,1,1,1,1,1,1,1,1
4,113,200,197,3,0,46,Administrative law and regulatory procedures,Rep.,D,FL,...,1,0,1,0,0,0,0,0,0,0


In [16]:
house_df.dtypes
# Need to create dummies for: Subject, Sponsor Title,  Sponsor Party, Sponsor State columns
# Need to bin Subject column before creating dummy vars.

Congress                              int64
Number of Cosponsors                  int64
Cosponsor Dems                        int64
Cosponsor Reps                        int64
Cosponsor Ind                         int64
Cosponsor States                      int64
Subject                              object
Sponsor Title                        object
Sponsor Party                        object
Sponsor State                        object
Month Introduced                      int64
Agriculture                           int64
Appropriations                        int64
Armed Services                        int64
Budget                                int64
Education and the Workforce           int64
Energy and Commerce                   int64
Ethics                                int64
Financial Services                    int64
Foreign Affairs                       int64
Homeland Security                     int64
House Administration                  int64
Judiciary                       

In [15]:
# Determine the number of unique values in each column.
house_df.nunique()


Congress                               6
Number of Cosponsors                 325
Cosponsor Dems                       236
Cosponsor Reps                       214
Cosponsor Ind                          2
Cosponsor States                      59
Subject                              768
Sponsor Title                          3
Sponsor Party                          3
Sponsor State                         56
Month Introduced                      12
Agriculture                            2
Appropriations                         2
Armed Services                         2
Budget                                 2
Education and the Workforce            2
Energy and Commerce                    2
Ethics                                 2
Financial Services                     2
Foreign Affairs                        2
Homeland Security                      2
House Administration                   2
Judiciary                              2
Natural Resources                      2
Oversight and Ac

In [ ]:
# Look at Subject value counts for binning
Subject_counts = house_df["Subject"].value_counts()
print(f'Count of values for Subject column: \n{Subject_counts}')

In [19]:
# Now change cutoff value to get  bins for Subject
# Cutoff value of 200 to bin: 
# use the variable name `Subject_types_to_replace`
Subject_types_to_replace = list(Subject_counts[Subject_counts<200].index)

# Replace in dataframe
for sub in Subject_types_to_replace:
    house_df['Subject'] = house_df['Subject'].replace(sub,"Other")

# Check to make sure binning was successful
house_df['Subject'].value_counts()
# 33 Bins

Other                                           15670
Health                                           2496
Armed Forces and National Security               2437
Taxation                                         1707
Administrative law and regulatory procedures     1580
Government Operations and Politics               1519
Crime and Law Enforcement                        1371
Education                                        1103
Transportation and Public Works                   884
Finance and Financial Sector                      880
Public Lands and Natural Resources                843
International Affairs                             825
Commerce                                          745
Congressional oversight                           656
Science, Technology, Communications               652
Immigration                                       600
Energy                                            570
Labor and Employment                              566
Agriculture and Food        

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
house_df = pd.get_dummies(house_df,dtype=float)
house_df.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,Agriculture,Appropriations,Armed Services,...,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VI,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,113,200,200,0,0,46,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,113,179,179,0,0,42,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,113,0,0,0,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,113,0,0,0,0,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,113,200,197,3,0,46,10,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Split our preprocessed data into our features and target arrays
y = house_df["bill_passed"].values
X = house_df.drop(["bill_passed"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Attempt #1


*   Total layers: 5 total
*   Activation function to for each layer is: relu, relu, tanh, tanh,  sigmoid
*   Number of neurons for each hidden layers: 9,7,7,5,1
*   Epochs: 100



In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="tanh"))

# Fourth hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 1161      
                                                                 
 dense_1 (Dense)             (None, 7)                 70        
                                                                 
 dense_2 (Dense)             (None, 7)                 56        
                                                                 
 dense_3 (Dense)             (None, 5)                 40        
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn_1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
954/954 [==============================] - 4s 2ms/step - loss: 0.1387 - accuracy: 0.9710
Epoch 2/100
954/954 [==============================] - 2s 2ms/step - loss: 0.1067 - accuracy: 0.9720
Epoch 3/100
954/954 [==============================] - 2s 2ms/step - loss: 0.1004 - accuracy: 0.9740
Epoch 4/100
954/954 [==============================] - 2s 2ms/step - loss: 0.0970 - accuracy: 0.9750
Epoch 5/100
954/954 [==============================] - 2s 2ms/step - loss: 0.0939 - accuracy: 0.9756
Epoch 6/100
954/954 [==============================] - 3s 3ms/step - loss: 0.0920 - accuracy: 0.9763
Epoch 7/100
954/954 [==============================] - 3s 3ms/step - loss: 0.0901 - accuracy: 0.9769
Epoch 8/100
954/954 [==============================] - 2s 2ms/step - loss: 0.0880 - accuracy: 0.9772
Epoch 9/100
954/954 [==============================] - 2s 2ms/step - loss: 0.0867 - accuracy: 0.9770
Epoch 10/100
954/954 [==============================] - 2s 2ms/step - loss: 0.0860 - accura

In [30]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {round(model_loss_1,4)}, Accuracy: {round(model_accuracy_1,4)}")

318/318 - 0s - loss: 0.1172 - accuracy: 0.9738 - 413ms/epoch - 1ms/step
Loss: 0.1172, Accuracy: 0.9738


### Save the model from optimization attempt 1: which had an accuracy of .97

In [31]:
# Export the model from optimization attempt 2 to HDF5 file
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/output"

# Save model 2, nn_2, and download a copy to local machine:
nn_1.save(f'{output_filepath}/house_model.h5')
files.download(f'{output_filepath}/house_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Senate Model

## Preprocessing:

In [34]:
# Glimpse senate data:
senate_df.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,S.,113,15,15,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Border security and unlawful immigration,...,0,0,0,0,0,0,0,0,0,1
1,S.,113,16,16,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Child health,...,0,0,0,0,0,0,0,0,0,1
2,S.,113,16,15,0,1,11,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,Child safety and welfare,...,0,0,0,0,0,0,0,0,0,1
3,S.,113,14,14,0,0,12,"Senate - Commerce, Science, and Transportation",Read twice and referred to the Committee on Co...,Academic performance and assessments,...,0,0,0,0,0,0,0,0,0,1
4,S.,113,31,30,0,1,24,Senate - Judiciary,Read twice and referred to the Committee on th...,Crime and Law Enforcement,...,0,0,0,0,0,0,0,0,0,1


In [35]:
# Check for NAs, duplicates and get the shape of the data:
print(f'The shape of the senate_df data is: {senate_df.shape}')
print(f'The number of NAs in the senate_df data: {senate_df.isnull().sum()}')
print(f'The duplicate rows of NAs in the senate_df data: {senate_df.duplicated().sum()}')
# There are 40 columns, 15593 rows, and no NAs or duplicates in the senate_df dataset.
# target is bill_passed

The shape of the senate_df data is: (15593, 40)
The number of NAs in the senate_df data: Bill Type                                       0
Congress                                        0
Number of Cosponsors                            0
Cosponsor Dems                                  0
Cosponsor Reps                                  0
Cosponsor Ind                                   0
Cosponsor States                                0
Committees                                    312
Latest Action                                   0
Subject                                       369
Sponsor Title                                   0
Sponsor Party                                   0
Sponsor State                                   0
Month Introduced                                0
Agriculture, Nutrition, and Forestry            0
Appropriations                                  0
Armed Services                                  0
Banking, Housing, and Urban Affairs             0
Budget     

In [36]:
# Numeric variable stats
senate_df.describe()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,"Agriculture, Nutrition, and Forestry",Appropriations,Armed Services,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
count,15593.0000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,...,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000,15593.000000
mean,114.9494,5.413006,2.804912,2.498044,0.110049,4.427500,5.422241,0.047393,0.027961,0.048419,...,0.020009,0.020009,0.035465,0.022382,0.020009,0.020009,0.020009,0.020009,0.029180,0.786827
std,1.4721,9.351292,6.024581,5.603372,0.355918,6.585518,3.225571,0.212485,0.164867,0.214657,...,0.140035,0.140035,0.184957,0.147927,0.140035,0.140035,0.140035,0.140035,0.168316,0.409562
min,113.0000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.0000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,115.0000,2.000000,1.000000,1.000000,0.000000,2.000000,5.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,116.0000,6.000000,3.000000,2.000000,0.000000,5.000000,7.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,118.0000,99.000000,55.000000,53.000000,3.000000,50.000000,12.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
# Get the column names:
list(senate_df.columns)


['Bill Type',
 'Congress',
 'Number of Cosponsors',
 'Cosponsor Dems',
 'Cosponsor Reps',
 'Cosponsor Ind',
 'Cosponsor States',
 'Committees',
 'Latest Action',
 'Subject',
 'Sponsor Title',
 'Sponsor Party',
 'Sponsor State',
 'Month Introduced',
 'Agriculture, Nutrition, and Forestry',
 'Appropriations',
 'Armed Services',
 'Banking, Housing, and Urban Affairs',
 'Budget',
 'Commerce, Science, and Transportation',
 'Energy and Natural Resources',
 'Environment and Public Works',
 'Finance',
 'Foreign Relations',
 'Health, Education, Labor, and Pensions',
 'Homeland Security and Governmental Affairs',
 'Judiciary',
 'Rules and Administration',
 'Small Business and Entrepreneurship',
 'Veterans Affairs',
 'International Narcotics Control',
 'Ethics',
 'Indian Affairs',
 'Intelligence',
 'Printing',
 'Taxation',
 'Library',
 'Economic',
 'bill_passed',
 'bill_referred_committee']

In [38]:
# Check out Subject column:
print(f'Count of values for Subject column: \n{senate_df["Subject"].value_counts()}')

Count of values for Subject column: 
Armed Forces and National Security              857
Administrative law and regulatory procedures    830
Health                                          763
Taxation                                        534
Crime and Law Enforcement                       411
                                               ... 
Germany                                           1
Israel                                            1
Nebraska                                          1
Federal Emergency Management Agency (FEMA)        1
Congressional districts and representation        1
Name: Subject, Length: 689, dtype: int64


In [39]:
# Model Target:
senate_df["bill_passed"].value_counts()
# 455 bills  in the dataset which originated from the House became law in the 113th-118th Congresses

0    15138
1      455
Name: bill_passed, dtype: int64

In [51]:
# Drop the non-beneficial columns: 'Unnamed' column.
senate_df = senate_df.drop(["Bill Type", "Committees", "Latest Action" ], axis='columns')
senate_df.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Subject,Sponsor Title,Sponsor Party,Sponsor State,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,113,15,15,0,0,12,Border security and unlawful immigration,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
1,113,16,16,0,0,12,Child health,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
2,113,16,15,0,1,11,Child safety and welfare,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
3,113,14,14,0,0,12,Academic performance and assessments,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
4,113,31,30,0,1,24,Crime and Law Enforcement,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1


In [43]:
senate_df.dtypes
# Need to create dummies for: Subject, Sponsor Title,  Sponsor Party, Sponsor State columns
# Need to bin Subject column before creating dummy vars.

Congress                                       int64
Number of Cosponsors                           int64
Cosponsor Dems                                 int64
Cosponsor Reps                                 int64
Cosponsor Ind                                  int64
Cosponsor States                               int64
Subject                                       object
Sponsor Title                                 object
Sponsor Party                                 object
Sponsor State                                 object
Month Introduced                               int64
Agriculture, Nutrition, and Forestry           int64
Appropriations                                 int64
Armed Services                                 int64
Banking, Housing, and Urban Affairs            int64
Budget                                         int64
Commerce, Science, and Transportation          int64
Energy and Natural Resources                   int64
Environment and Public Works                  

In [44]:
# Determine the number of unique values in each column.
senate_df.nunique()

Congress                                        6
Number of Cosponsors                           86
Cosponsor Dems                                 53
Cosponsor Reps                                 53
Cosponsor Ind                                   4
Cosponsor States                               51
Subject                                       689
Sponsor Title                                   1
Sponsor Party                                   3
Sponsor State                                  50
Month Introduced                               12
Agriculture, Nutrition, and Forestry            2
Appropriations                                  2
Armed Services                                  2
Banking, Housing, and Urban Affairs             2
Budget                                          2
Commerce, Science, and Transportation           2
Energy and Natural Resources                    2
Environment and Public Works                    2
Finance                                         2


In [52]:
# Look at Subject value counts for binning
Subject_counts_sen = senate_df["Subject"].value_counts()
print(f'Count of values for Subject column: \n{Subject_counts_sen}')

Count of values for Subject column: 
Armed Forces and National Security              857
Administrative law and regulatory procedures    830
Health                                          763
Taxation                                        534
Crime and Law Enforcement                       411
                                               ... 
Germany                                           1
Israel                                            1
Nebraska                                          1
Federal Emergency Management Agency (FEMA)        1
Congressional districts and representation        1
Name: Subject, Length: 689, dtype: int64


In [55]:
# Now change cutoff value to get  bins for Subject
# Cutoff value of 100 to bin: 
# use the variable name `Subject_types_to_replace_sen`
Subject_types_to_replace_sen = list(Subject_counts_sen[Subject_counts_sen<100].index)

# Replace in dataframe
for sub in Subject_types_to_replace_sen:
    senate_df['Subject'] = senate_df['Subject'].replace(sub,"Other")

# Check to make sure binning was successful
senate_df['Subject'].value_counts()
# 27 Bins

Other                                           7401
Armed Forces and National Security               857
Administrative law and regulatory procedures     830
Health                                           763
Taxation                                         534
Crime and Law Enforcement                        411
Government Operations and Politics               394
Public Lands and Natural Resources               325
Congressional oversight                          313
International Affairs                            310
Finance and Financial Sector                     291
Education                                        280
Transportation and Public Works                  262
Commerce                                         253
Science, Technology, Communications              204
Agriculture and Food                             192
Energy                                           190
Advisory bodies                                  176
Appropriations                                

In [56]:
# Convert categorical data to numeric with `pd.get_dummies`
senate_df = pd.get_dummies(senate_df,dtype=float)
senate_df.head()
# Creates a df of 114 columns

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,"Agriculture, Nutrition, and Forestry",Appropriations,Armed Services,...,Sponsor State_SD,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,113,15,15,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,113,16,16,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,113,16,15,0,1,11,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,113,14,14,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,113,31,30,0,1,24,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# Split our preprocessed data into our features and target arrays
y = senate_df["bill_passed"].values
X = senate_df.drop(["bill_passed"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Attempt #1


*   Total layers: 5 total
*   Activation function to for each layer is: relu, relu, tanh, tanh,  sigmoid
*   Number of neurons for each hidden layers: 9,7,7,5,1
*   Epochs: 100

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5

nn_sen = tf.keras.models.Sequential()

# First hidden layer
nn_sen.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="tanh"))

# Fourth hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_sen.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_sen.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 9)                 1026      
                                                                 
 dense_6 (Dense)             (None, 7)                 70        
                                                                 
 dense_7 (Dense)             (None, 7)                 56        
                                                                 
 dense_8 (Dense)             (None, 5)                 40        
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,198
Trainable params: 1,198
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Compile the model
nn_sen.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [61]:
# Train the model
fit_model = nn_sen.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
366/366 [==============================] - 3s 3ms/step - loss: 0.2256 - accuracy: 0.9525
Epoch 2/100
366/366 [==============================] - 1s 3ms/step - loss: 0.1215 - accuracy: 0.9710
Epoch 3/100
366/366 [==============================] - 1s 2ms/step - loss: 0.1045 - accuracy: 0.9711
Epoch 4/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0897 - accuracy: 0.9709
Epoch 5/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0820 - accuracy: 0.9714
Epoch 6/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0778 - accuracy: 0.9714
Epoch 7/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0734 - accuracy: 0.9720
Epoch 8/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0706 - accuracy: 0.9720
Epoch 9/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0684 - accuracy: 0.9727
Epoch 10/100
366/366 [==============================] - 1s 2ms/step - loss: 0.0667 - accura

In [62]:
# Evaluate the Senate model using the test data:
model_loss_1, model_accuracy_1 = nn_sen.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {round(model_loss_1,4)}, Accuracy: {round(model_accuracy_1,4)}")

122/122 - 0s - loss: 0.1850 - accuracy: 0.9515 - 343ms/epoch - 3ms/step
Loss: 0.185, Accuracy: 0.9515


In [63]:
# Export the model from optimization attempt 2 to HDF5 file
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/output"

# Save model 2, nn_2, and download a copy to local machine:
nn_1.save(f'{output_filepath}/senate_model.h5')
files.download(f'{output_filepath}/senate_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>